In [1]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import time

In [6]:
def get_team_df(dict_correspondances, countries):
    player_name, player_country, player_position, player_age, player_description, player_link = [], [], [], [], [], []
    browser = webdriver.Firefox()
    browser.get("https://www.fifa.com/worldcup/teams/")
    time.sleep(1)
    
    for country in countries:
        element = browser.find_elements_by_xpath("//div[@class='fi-team-card__flag']")[dict_correspondances[country]]
        browser.execute_script("arguments[0].click();", element)
        time.sleep(1)

        for p in range(23):
            try:
                player = browser.find_elements_by_xpath("//div[@class='fi-p']/a[@class='fi-p--link']")[p]
                new_url = player.get_attribute('href')
                browser.get(new_url)
                time.sleep(1)
                player_name.append(browser.find_element_by_xpath("//div[@class='fi-p__name']").text)
                print(player_name[-1])
                player_country.append(country)
                player_position.append(browser.find_element_by_xpath("//div[@class='fi-p__role']").text)
                player_age.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-number__number']").text)
                try:
                    player_description.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-text']").text)
                except NoSuchElementException:
                    player_description.append("")
                my_img = browser.find_elements_by_tag_name('image')
                for image in my_img:
                    link = image.get_attribute('xlink:href')
                    player_link.append(link)
                browser.back()
                time.sleep(1)
            except IndexError:
                print()

        browser.get("https://www.fifa.com/worldcup/teams/")
        time.sleep(2)
    
    df = pd.DataFrame({"Name":player_name, "Country":player_country, "Position":player_position, "Age":player_age, 
                   "Description":player_description, "Link":player_link})
    df["Firstname"] = df["Name"].apply(lambda x: x.split()[0].capitalize())
    df["Lastname"] = df["Name"].apply(lambda x: "".join(x.split()[1:]).capitalize())
    return df

dict_correspondances = {"Australia": 0, "Belgium": 2, "Brazil": 3,
                        "Colombia": 4, "Costa": 5, "Croatia": 6,
                        "Denmark": 7, "Egypt": 8, "England": 9,
                        "France": 10, "Germany": 11, "Iran": 13,
                        "Morocco": 17, "Peru": 20, "Poland": 21,
                        "Portugal": 22, "Russia": 23, "Arabia": 24,
                        "Serbia": 26, "Spain": 27, "Sweden": 28,
                        "Switzerland": 29, "Tunisia": 30}

df = get_team_df(dict_correspondances, dict_correspondances.keys())

NAHUEL GUZMAN
FRANCO ARMANI
WILFREDO CABALLERO
GABRIEL MERCADO
NICOLAS TAGLIAFICO
CRISTIAN ANSALDI
FEDERICO FAZIO
MARCOS ACUNA
JAVIER MASCHERANO
MARCOS ROJO
NICOLAS OTAMENDI
EDUARDO SALVIO
LUCAS BIGLIA
EVER BANEGA
ANGEL DI MARIA
MAXIMILIANO MEZA
ENZO PEREZ
GIOVANI LO CELSO
CRISTIAN PAVON
GONZALO HIGUAIN
LIONEL MESSI
SERGIO AGUERO
PAULO DYBALA
THIBAUT COURTOIS
SIMON MIGNOLET
KOEN CASTEELS
TOBY ALDERWEIRELD
THOMAS VERMAELEN
VINCENT KOMPANY
JAN VERTONGHEN
THOMAS MEUNIER
DEDRYCK BOYATA
LEANDER DENDONCKER
AXEL WITSEL
KEVIN DE BRUYNE
MAROUANE FELLAINI
YANNICK CARRASCO
THORGAN HAZARD
YOURI TIELEMANS
MOUSSA DEMBELE
NACER CHADLI
ROMELU LUKAKU
EDEN HAZARD
DRIES MERTENS
ADNAN JANUZAJ
MICHY BATSHUAYI
ALISSON
CASSIO
EDERSON
THIAGO SILVA
MIRANDA
PEDRO GEROMEL
FILIPE LUIS
MARCELO
MARQUINHOS
DANILO
FAGNER
CASEMIRO
RENATO AUGUSTO
PHILIPPE COUTINHO
PAULINHO
FERNANDINHO
FRED
WILLIAN
DOUGLAS COSTA
GABRIEL JESUS
NEYMAR
ROBERTO FIRMINO
TAISON
DAVID OSPINA
CAMILO VARGAS
JOSE CUADRADO
CRISTIAN ZAPATA
OSCAR MU

In [8]:
df.shape

(520, 8)

In [31]:
dict_trig = {"Australia": 'AUS', "Belgium": 'BEL', "Brazil": 'BRA',
            "Colombia": 'COL', "Costa": 'CRI', "Croatia": 'HRV',
            "Denmark": 'DEN', "Egypt": 'EGY', "England": 'ENG',
            "France": 'FRA', "Germany": 'GER', "Iran": 'IRN',
            "Morocco": 'MAR', "Peru": 'PER', "Poland": 'POL',
            "Portugal": 'POR', "Russia": 'RUS', "Arabia": 'ARA',
            "Serbia": 'SER', "Spain": 'SPA', "Sweden": 'SWE',
            "Switzerland": 'SUI', "Tunisia": 'TUN','United_States':'USA'}


def get_lastname(text):
    text = text.split()
    if len(text) == 1:
        return text[0].capitalize()
    else:
        return ''.join(text[1:]).capitalize()
    
def get_trigramme(text):
    return dict_trig[text]

df = pd.read_csv("df_full.csv")
df["Lastname"] = df["Name"].apply(lambda x: get_lastname(x))
df["Trigramme"] = df["Country"].apply(lambda x: get_trigramme(x))
df.head()

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
0,32,Australia,"Solidly built and a born competitor, Nahuel Gu...",https://api.fifa.com/api/v1/picture/players/20...,NAHUEL GUZMAN,Goalkeeper,Nahuel,Guzman,AUS
1,31,Australia,Franco Armani owes his place at Russia 2018 to...,https://api.fifa.com/api/v1/picture/players/20...,FRANCO ARMANI,Goalkeeper,Franco,Armani,AUS
2,36,Australia,"Willy Caballero is an experienced, unfussy goa...",https://api.fifa.com/api/v1/picture/players/20...,WILFREDO CABALLERO,Goalkeeper,Wilfredo,Caballero,AUS
3,31,Australia,NaN,https://api.fifa.com/api/v1/picture/players/20...,GABRIEL MERCADO,Defender,Gabriel,Mercado,AUS
4,25,Australia,"“A natural full-back,” remarked Jorge Sampaoli...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAS TAGLIAFICO,Defender,Nicolas,Tagliafico,AUS


In [37]:
df["Lastname"].value_counts()[:16]

Silva        5
Fernandes    4
Rodriguez    3
Hawsawi      3
Miranchuk    2
Amrabat      2
Jorgensen    2
Cuadrado     2
Dembele      2
Kalinic      2
Costa        2
Guzman       2
Sanchez      2
Hazard       2
Ramos        2
Cheshmi      1
Name: Lastname, dtype: int64

In [ ]:
df.loc[df.Name == "BERNARDO SILVA", 'Lastname'] = "Bernardo"
df.loc[df.Name == "ADRIEN SILVA", 'Lastname'] = "Adrien"
df.loc[df.Name == "ANDRE SILVA", 'Lastname'] = "Andre"

In [ ]:
df.loc[df.Name == "BRUNO FERNANDES", 'Lastname'] = "Bruno"

In [69]:
df[df["Lastname"] == "Hawsawi"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
391,34,Arabia,Osama Hawsawi is among the most experienced pl...,https://api.fifa.com/api/v1/picture/players/20...,OSAMA HAWSAWI,Defender,Osama,Hawsawi,ARA
395,26,Arabia,Motaz Hawsawi bolstered the Saudi Arabia defen...,https://api.fifa.com/api/v1/picture/players/20...,MOTAZ HAWSAWI,Defender,Motaz,Hawsawi,ARA
519,32,Arabi,NaN,https://api.fifa.com/api/v1/picture/players/20...,OMAR HAWSAWI,Defender,Omar,Hawsawi,ARA


In [71]:
df.loc[df.Name == "OMAR HAWSAWI", 'Lastname'] = "Omar"

In [73]:
df.loc[df.Name == "MOTAZ HAWSAWI", 'Lastname'] = "Motaz"

In [74]:
df[df["Lastname"] == "Hawsawi"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
391,34,Arabia,Osama Hawsawi is among the most experienced pl...,https://api.fifa.com/api/v1/picture/players/20...,OSAMA HAWSAWI,Defender,Osama,Hawsawi,ARA


In [75]:
df[df["Lastname"] == "Miranchuk"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
379,22,Russia,"At the relatively young age of 22, Aleksey Mir...",https://api.fifa.com/api/v1/picture/players/20...,ALEXEY MIRANCHUK,Midfielder,Alexey,Miranchuk,RUS
380,22,Russia,A unique changing of the guard has recently ta...,https://api.fifa.com/api/v1/picture/players/20...,ANTON MIRANCHUK,Midfielder,Anton,Miranchuk,RUS


In [78]:
df.loc[df.Name == "ALEXEY MIRANCHUK", 'Lastname'] = "Alexey"

In [79]:
df[df["Lastname"] == "Amrabat"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
287,31,Morocco,"Like his younger brother, Sofyan, Nordin Amrab...",https://api.fifa.com/api/v1/picture/players/20...,NOUREDDINE AMRABAT,Midfielder,Noureddine,Amrabat,MAR
289,21,Morocco,"The younger sibling of Nordin, also a Morocco ...",https://api.fifa.com/api/v1/picture/players/20...,SOFYAN AMRABAT,Midfielder,Sofyan,Amrabat,MAR


In [80]:
df.loc[df.Name == "NOUREDDINE AMRABAT", 'Lastname'] = "Noureddine"

In [81]:
df[df["Lastname"] == "Amrabat"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
289,21,Morocco,"The younger sibling of Nordin, also a Morocco ...",https://api.fifa.com/api/v1/picture/players/20...,SOFYAN AMRABAT,Midfielder,Sofyan,Amrabat,MAR


In [82]:
df[df["Lastname"] == "Jorgensen"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
145,28,Denmark,A centre-back who can provide an aerial presen...,https://api.fifa.com/api/v1/picture/players/20...,MATHIAS JORGENSEN,Defender,Mathias,Jorgensen,DEN
154,27,Denmark,"A key member of Denmark’s attack, Nicolai Jorg...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAI JORGENSEN,Forward,Nicolai,Jorgensen,DEN


In [83]:
df.loc[df.Name == "MATHIAS JORGENSEN", 'Lastname'] = "Mathias"

In [84]:
df[df["Lastname"] == "Jorgensen"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
154,27,Denmark,"A key member of Denmark’s attack, Nicolai Jorg...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAI JORGENSEN,Forward,Nicolai,Jorgensen,DEN


In [85]:
df[df["Lastname"] == "Cuadrado"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
71,33,Colombia,"A product of Millonarios’ youth system, and wi...",https://api.fifa.com/api/v1/picture/players/20...,JOSE CUADRADO,Goalkeeper,Jose,Cuadrado,COL
83,30,Colombia,"A Colombia international since 2010, Juan Cuad...",https://api.fifa.com/api/v1/picture/players/20...,JUAN CUADRADO,Midfielder,Juan,Cuadrado,COL


In [86]:
df.loc[df.Name == "JOSE CUADRADO", 'Lastname'] = "Jose"

In [87]:
df[df["Lastname"] == "Cuadrado"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
83,30,Colombia,"A Colombia international since 2010, Juan Cuad...",https://api.fifa.com/api/v1/picture/players/20...,JUAN CUADRADO,Midfielder,Juan,Cuadrado,COL


In [89]:
df[df["Lastname"] == "Kalinic"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
116,28,Croatia,After Stipe Pletikosa retired from internation...,https://api.fifa.com/api/v1/picture/players/20...,LOVRE KALINIC,Goalkeeper,Lovre,Kalinic,HRV
134,30,Croatia,Nikola Kalinic is one of a handful of players ...,https://api.fifa.com/api/v1/picture/players/20...,NIKOLA KALINIC,Forward,Nikola,Kalinic,HRV


In [90]:
df.loc[df.Name == "LOVRE KALINIC", 'Lastname'] = "Lovre"

In [91]:
df[df["Lastname"] == "Kalinic"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
134,30,Croatia,Nikola Kalinic is one of a handful of players ...,https://api.fifa.com/api/v1/picture/players/20...,NIKOLA KALINIC,Forward,Nikola,Kalinic,HRV


In [94]:
df[df["Lastname"] == "Sanchez"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
78,22,Colombia,"A predominantly right-footed centre-half, Davi...",https://api.fifa.com/api/v1/picture/players/20...,DAVINSON SANCHEZ,Defender,Davinson,Sanchez,COL
80,32,Colombia,"A veritable cornerstone, holding midfielder Ca...",https://api.fifa.com/api/v1/picture/players/20...,CARLOS SANCHEZ,Midfielder,Carlos,Sanchez,COL


In [95]:
df.loc[df.Name == "DAVINSON SANCHEZ", 'Lastname'] = "Davinson"

In [96]:
df[df["Lastname"] == "Sanchez"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
80,32,Colombia,"A veritable cornerstone, holding midfielder Ca...",https://api.fifa.com/api/v1/picture/players/20...,CARLOS SANCHEZ,Midfielder,Carlos,Sanchez,COL


In [97]:
df[df["Lastname"] == "Hazard"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
37,25,Belgium,Having spent a long time trying to avoid compa...,https://api.fifa.com/api/v1/picture/players/20...,THORGAN HAZARD,Midfielder,Thorgan,Hazard,BEL
42,27,Belgium,"Comfortable on the ball, an elusive dribbler a...",https://api.fifa.com/api/v1/picture/players/20...,EDEN HAZARD,Forward,Eden,Hazard,BEL


In [98]:
df.loc[df.Name == "THORGAN HAZARD", 'Lastname'] = "Thorgan"

In [99]:
df[df["Lastname"] == "Hazard"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
42,27,Belgium,"Comfortable on the ball, an elusive dribbler a...",https://api.fifa.com/api/v1/picture/players/20...,EDEN HAZARD,Forward,Eden,Hazard,BEL


In [100]:
df[df["Lastname"] == "Ramos"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
303,29,Peru,"Nicknamed _La Sombra_ (‘The Shadow’), Christia...",https://api.fifa.com/api/v1/picture/players/20...,CHRISTIAN RAMOS,Defender,Christian,Ramos,PER
440,32,Spain,"One of the world’s finest central defenders, a...",https://api.fifa.com/api/v1/picture/players/20...,SERGIO RAMOS,Defender,Sergio,Ramos,SPA


In [101]:
df.to_csv("df_full_Maxime.csv",index=False)

In [102]:
df_maxime = df.rename(columns={"Trigramme":"Code","Position":"Post"})

In [103]:
df_maxime

,Age,Country,Description,Link,Name,Post,Firstname,Lastname,Code
0,32,Australia,"Solidly built and a born competitor, Nahuel Gu...",https://api.fifa.com/api/v1/picture/players/20...,NAHUEL GUZMAN,Goalkeeper,Nahuel,Guzman,AUS
1,31,Australia,Franco Armani owes his place at Russia 2018 to...,https://api.fifa.com/api/v1/picture/players/20...,FRANCO ARMANI,Goalkeeper,Franco,Armani,AUS
2,36,Australia,"Willy Caballero is an experienced, unfussy goa...",https://api.fifa.com/api/v1/picture/players/20...,WILFREDO CABALLERO,Goalkeeper,Wilfredo,Caballero,AUS
3,31,Australia,NaN,https://api.fifa.com/api/v1/picture/players/20...,GABRIEL MERCADO,Defender,Gabriel,Mercado,AUS
4,25,Australia,"“A natural full-back,” remarked Jorge Sampaoli...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAS TAGLIAFICO,Defender,Nicolas,Tagliafico,AUS
5,31,Australia,Since winning his first senior cap under Diego...,https://api.fifa.com/api/v1/picture/players/20...,CRISTIAN ANSALDI,Defender,Cristian,Ansaldi,AUS
6,31,Australia,NaN,https://api.fifa.com/api/v1/picture/players/20...,FEDERICO FAZIO,Defender,Federico,Fazio,AUS
7,26,Australia,Though the left-footed Marcos Acuna has spent ...,https://api.fifa.com/api/v1/picture/players/20...,MARCOS ACUNA,Defender,Marcos,Acuna,AUS
8,34,Australia,_El_ _Jefecito _(“The Little Chief”) has lost ...,https://api.fifa.com/api/v1/picture/players/20...,JAVIER MASCHERANO,Defender,Javier,Mascherano,AUS
9,28,Australia,"Now an experienced performer, Marcos Rojo is a...",https://api.fifa.com/api/v1/picture/players/20...,MARCOS ROJO,Defender,Marcos,Rojo,AUS


In [104]:
df_maxime.to_csv("data_worldcup_new.csv",index=False)

In [ ]:
df_maxime[df_maxime["Country"] == "Russia"]

In [107]:
def image_downloader(name, team, link):
    with open(team.lower() + '/' + name.capitalize() + '.jpg', 'wb') as handle:
            response = requests.get(link, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)

for n, t, l in zip(df_maxime[df_maxime["Country"] == "Arabia"]["Lastname"].tolist(), df_maxime[df_maxime["Country"] == "Arabia"]["Country"].tolist(), df_maxime[df_maxime["Country"] == "Arabia"]["Link"].tolist()):
    image_downloader(n, t, l)